# Packages

In [ ]:
import cv2

from google.colab import files
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from IPython.display import clear_output, HTML
clear_output()

# Prep

In [ ]:
path = '/usr/local/share/jupyter/nbextensions/google.colab'
!cp -r {path}/* .
!rm -r {path}
!ln -s /content {path}
# change base tag

def change_base_url():
  eval_js("""
  var base = document.createElement('base')
  base.href = 'https://localhost:8080/nbextensions/google.colab/'
  document.head.prepend(base)
  """)
# make it run automatically in every cell
get_ipython().events.register('pre_run_cell', change_base_url)

fpath_img = 'od_sample4.jpeg'
!gdown 12wANIcnjo2RGPNqTu07hQa0-9qR4posR -O "$fpath_img"

clear_output()

# Object Detection

In [ ]:
#@title Template Image
html_code = HTML('''
<!-- 讀取tfjs依賴套件 -->
<script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs@latest/dist/tf.min.js"></script>
<!-- 讀取物件偵測套件 -->
<script src="https://cdn.jsdelivr.net/npm/@tensorflow-models/coco-ssd"> </script>

<!-- 呈現結果的canvas -->
<canvas></canvas>
<script>
// 宣告canvas 物件
var canvas = document.querySelector('canvas')
// canvas 繪製物件
var ctx = canvas.getContext('2d')

// 宣告js影像物件
var myImage = new Image();
myImage.src = "od_sample4.jpeg"; // 範例影像路徑
myImage.addEventListener("load", loadImage, false);// 一旦成功讀取影像，觸發loadImage

// 觸發函示
function loadImage(e) {
  // 範例影像大小
  img_width = myImage.width;
  img_height = myImage.height;

  // 調整canvas大小
  canvas.width = img_width;
  canvas.height = img_height;

  ctx.strokeStyle = "red"; // 物件框顏色
  ctx.font = "30px Arial"; // 物件類別註解文字大小
  ctx.drawImage(myImage, 0, 0, img_width, img_height);// 繪製底圖

  // 讀取模型
  cocoSsd.load().then(model => {
    // 模型分析推論
    model.detect(canvas).then(predictions => {
      // 繪製分析結果
      for (var i = 0; i < predictions.length; i++) {
          console.log(predictions[i]);

          // 物件框
          var x = predictions[i].bbox[0];
          var y = predictions[i].bbox[1];
          var w = predictions[i].bbox[2];
          var h = predictions[i].bbox[3];
          ctx.strokeRect(x, y, w, h);

          // 物件類別文字
          ctx.fillText(predictions[i].class, x, y+30);
      }
    });
  });
}
</script>
''')

html_code

In [ ]:
#@title WebCam Image
html_code3 = HTML('''
<!-- 讀取tfjs依賴套件 -->
<script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs"> </script>
<!-- 讀取物件偵測套件 -->
<script src="https://cdn.jsdelivr.net/npm/@tensorflow-models/coco-ssd"> </script>
<!-- 用於呈現分析結果canvas -->
<canvas id='canvas1' height="0" width="0"></canvas>

<script>
// 宣告canvas 物件
const canvas = document.getElementById('canvas1');
// 宣告canvas 繪圖物件
var ctx = canvas.getContext('2d')

// webcam 攝影
async function takePhoto() {
  // 讀取模型
  const model = await cocoSsd.load();

  const div = document.createElement('div'); // 空行
  const capture = document.createElement('button'); // 攝影按鈕
  capture.textContent = 'Capture'; // 攝影按鈕文字
  div.appendChild(capture);

  const video = document.createElement('video'); // 宣告影片物件
  video.style.display = 'block';
  const stream = await navigator.mediaDevices.getUserMedia({video: true}); // 宣告串流物件

  document.body.appendChild(div);
  div.appendChild(video);
  video.srcObject = stream; // 使影片物件顯示串流內容
  await video.play();

  // 調整Video Element 大小
  google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

  // 等待點擊攝影按鈕
  await new Promise((resolve) => capture.onclick = resolve);

  // canvas 繪製底圖
  canvas.width = video.videoWidth;
  canvas.height = video.videoHeight;
  ctx.drawImage(video, 0, 0);

  // 停止 webcamera
  stream.getVideoTracks()[0].stop();
  div.remove();

  // 模型推論分析
  const predictions = await model.detect(canvas);

  // 繪製分析結果
  ctx.strokeStyle = "red";
  ctx.font = "30px Arial";
  for (var i = 0; i < predictions.length; i++) {
      console.log(predictions[i]);

      // 物件框
      var x = predictions[i].bbox[0];
      var y = predictions[i].bbox[1];
      var w = predictions[i].bbox[2];
      var h = predictions[i].bbox[3];
      ctx.strokeRect(x, y, w, h);

      // 物件類別文字
      ctx.fillText(predictions[i].class, x, y+30);
  }
}

takePhoto()
</script>
''')

html_code3

In [ ]:
#@title WebCam Video
html_code4 = HTML('''
<head>
  <!-- 讀取tfjs依賴套件 -->
  <script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs"> </script>
  <!-- 讀取物件偵測套件 -->
  <script src="https://cdn.jsdelivr.net/npm/@tensorflow-models/coco-ssd"> </script>

  <!-- 隱藏html video -->
  <style>
      video{
          visibility: hidden;
      }
  </style>
</head>

<!-- 呈現結果的canvas -->
<canvas id="detect_result"></canvas>
<!-- 串流video -->
<video autoplay playsinline muted id="webcam" ></video>

<script>
async function app() {
  // 讀取模型
  const model = await cocoSsd.load();

  const webcamElement = document.getElementById('webcam'); // 宣告影片物件

  // 宣告canvas物件
  const canvas = document.getElementById('detect_result');
  const ctx = canvas.getContext('2d');

  let showResult = async function () {
    // 模型推論分析
    const predictions = await model.detect(canvas);

    // canvas 繪製底圖
    canvas.width = webcamElement.videoWidth;
    canvas.height = webcamElement.videoHeight;
    ctx.drawImage(webcamElement, 0, 0);

    // 繪製分析結果
    ctx.strokeStyle = "red";
    ctx.font = "30px Arial";
    for (var i = 0; i < predictions.length; i++) {
        console.log(predictions[i]);

        // 物件框
        var x = predictions[i].bbox[0];
        var y = predictions[i].bbox[1];
        var w = predictions[i].bbox[2];
        var h = predictions[i].bbox[3];
        ctx.strokeRect(x, y, w, h);

        // 物件類別文字
        ctx.fillText(predictions[i].class, x, y+30);
    }

    // 設定每偵顯示秒數
    setTimeout(function () {
        showResult();
    }, 300);
  }
  let setupWebcam = function () {
      return new Promise((resolve, reject) => {
          const navigatorAny = navigator;
          navigator.getUserMedia = navigator.getUserMedia ||
              navigatorAny.webkitGetUserMedia || navigatorAny.mozGetUserMedia ||
              navigatorAny.msGetUserMedia;
          if (navigator.getUserMedia) {
              navigator.getUserMedia({
                      video: true
                  },
                  (stream) => {
                      webcamElement.srcObject = stream;
                      webcamElement.addEventListener('loadeddata', () => resolve(),
                          false);
                  },
                  (err) => reject(err));
          } else {
              reject("getUserMedia failed");
          }
      });
  }
  setupWebcam().then(
      () => {
          showResult();
      },
      (err) => {
          console.log(err);
      }
  )
}
app();
</script>
''')

html_code4